In [7]:
import pandas as pd
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'data')))

# importing the JSON data
from data import data

# Importing necessary libraries for data preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Importing text preprocessing class from sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
# Importing Pipeline and ColumnTransformer for preprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Loding data

In [2]:
data = pd.read_csv('../data/raw/data.csv')

In [5]:
data.head()

,user_goal,habit_log,habit_type,sentiment
0,I want to learn how to play the guitar.,I practiced guitar for 30 minutes today.,Learning,Positive
1,I want to learn how to play the guitar.,I watched TV instead of practicing guitar.,Distraction,Negative
2,I want to learn a new language.,I practiced Spanish for 15 minutes today.,Learning,Positive
3,I want to learn a new language.,I didn't practice my language skills this week.,Learning,Negative
4,I want to learn photography.,I took photos in manual mode and watched a tut...,Learning,Positive


# Cleaning and Preprocessing

In [4]:
data.drop(columns=["aligned"], inplace=True)

In [6]:
data["habit_type"].unique()

array(['Learning', 'Distraction', 'Cooking', 'Fitness', 'Nutrition',
       'Sleep', 'Mindfulness', 'Organization', 'Relationships',
       'Community Service', 'Finance', 'Communication', 'Reading',
       'Programming', 'Planning', 'Sustainability', 'Health', 'Gardening',
       'Maintenance', 'Security', 'Wellbeing', 'Lifestyle', 'Safety',
       'Creativity'], dtype=object)

In [ ]:
# # Seperating data for two different ML models
# data_for_habit_type = data.drop(columns=["sentiment"])
# data_for_sentiment = data.drop(columns=["habit_type"])